In [2]:
import pandas as pd

stocks_info = pd.read_csv('../sources/s&p500.csv')
stocks_symbol = stocks_info['Symbol']

In [3]:
import snscrape.modules.twitter as sntwitter

# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for s in stocks_symbol[0:1]:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('${} (from:jimcramer) since:1990-01-01 until:2023-12-31'.format(s)).get_items()):
        tweets_list.append([tweet.date, tweet.content.replace('\n', ''), tweet.retweetedTweet, s])
    print(s)
    
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text', 'Retweeted', 'Symbol'])

/var/folders/xt/tq9cyssn6k51lc2g0m8nrppc0000gn/T/ipykernel_9870/3474067894.py:9: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets_list.append([tweet.date, tweet.content.replace('\n', ''), tweet.retweetedTweet, s])


MMM


In [4]:
tweets_df

,Datetime,Text,Retweeted,Symbol
0,2023-01-25 03:05:45+00:00,To me the worst quarter of the season so far i...,None,MMM
1,2021-11-30 14:22:02+00:00,"$MRNA, $SQ, $F, $MMM &amp; more… all covered i...",None,MMM
2,2020-09-24 08:36:51+00:00,If you look at the https://t.co/NS3syOn64o you...,None,MMM
3,2020-08-27 08:46:42+00:00,thank you to Mike Roman and the great folks at...,None,MMM
4,2020-01-28 17:44:08+00:00,the stock needs to see that return.. as this i...,None,MMM
5,2018-11-14 16:01:06+00:00,"People don't want a buyback from $MMM, they wa...",None,MMM
6,2018-01-19 08:55:08+00:00,I have always loved Minneapolis and want to do...,None,MMM
7,2017-10-24 18:12:00+00:00,Today’s numbers show why $MMM is a core holdin...,None,MMM
8,2017-10-24 11:37:51+00:00,Earnings--it was always earnings. That's the g...,None,MMM
9,2016-08-17 14:32:58+00:00,thanks for the kudos. $MMM has done a fabulous...,None,MMM


In [5]:
tweets_df.to_csv('../sources/data.csv', index=False)

In [6]:
df = pd.read_csv('../sources/data.csv')

In [7]:
df["Retweeted"].value_counts()

Series([], Name: Retweeted, dtype: int64)

In [8]:
from datetime import datetime, timedelta
import yfinance as yf

def addDate(date, day):
    date1 = datetime.strptime(date, '%Y-%m-%d').date()
    date2 = date1 + timedelta(days=day)
    return str(date2)

def getReturn(old_price, new_price):
    return (new_price - old_price) / old_price

def yfSearch(row, days):
    ticker = yf.Tickers(row['Symbol'])
    adj = 1
    while True:
        try:
            return ticker.tickers[row['Symbol']].history(start=addDate(row['Datetime'][:10], days), 
                                                         end=addDate(row['Datetime'][:10], days+adj), 
                                                         raise_errors=True)['Close'][1]
        except:
            adj += 1




In [9]:
df['price'] = df.apply(lambda row: yfSearch(row, 0), axis=1)
print("p")
df['30_days_price'] = df.apply(lambda row: yfSearch(row, 30), axis=1)
print("p1")
df['90_days_price'] = df.apply(lambda row: yfSearch(row, 90), axis=1)
print("p2")
df['180_days_price'] = df.apply(lambda row: yfSearch(row, 180), axis=1)
print("p3")

p
p1


In [ ]:
df

,Datetime,Text,Retweeted,Symbol,price
0,2023-01-25 03:05:45+00:00,To me the worst quarter of the season so far i...,NaN,MMM,112.066200
1,2021-11-30 14:22:02+00:00,"$MRNA, $SQ, $F, $MMM &amp; more… all covered i...",NaN,MMM,161.645111
2,2020-09-24 08:36:51+00:00,If you look at the https://t.co/NS3syOn64o you...,NaN,MMM,145.812012
3,2020-08-27 08:46:42+00:00,thank you to Mike Roman and the great folks at...,NaN,MMM,150.715775
4,2020-01-28 17:44:08+00:00,the stock needs to see that return.. as this i...,NaN,MMM,143.324203
5,2018-11-14 16:01:06+00:00,"People don't want a buyback from $MMM, they wa...",NaN,MMM,174.188675
6,2018-01-19 08:55:08+00:00,I have always loved Minneapolis and want to do...,NaN,MMM,206.189270
7,2017-10-24 18:12:00+00:00,Today’s numbers show why $MMM is a core holdin...,NaN,MMM,197.173798
8,2017-10-24 11:37:51+00:00,Earnings--it was always earnings. That's the g...,NaN,MMM,197.173798
9,2016-08-17 14:32:58+00:00,thanks for the kudos. $MMM has done a fabulous...,NaN,MMM,145.035568
